# Linear Algebra for AI/ML - Part 4: Projections, Orthogonality & Eigenvalues

This notebook covers projection matrices, orthogonality concepts, and eigenvalue decomposition.

**Prerequisites:** Complete Parts 1-3 first.

In [1]:
"""
Setup: Import Required Libraries
"""
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

np.set_printoptions(precision=4, suppress=True)
print(f"NumPy version: {np.__version__}")

NumPy version: 2.2.6


## 24. Projection Matrix

A projection matrix P projects vectors onto a subspace.

**Properties:**
1. P² = P (idempotent)
2. Pᵀ = P (symmetric)

**Formula:** P = A(AᵀA)⁻¹Aᵀ

**ML Application:** Linear regression, PCA, orthogonal projections.

In [2]:
"""
Projection Matrix

Purpose:
Projects any vector onto a subspace (e.g., line, plane)
Keeps the component in the subspace, discards the perpendicular part

Formula:
For subspace spanned by columns of A:
    P = A(AᵀA)⁻¹Aᵀ

Properties:
1. P² = P (applying twice = applying once)
2. Pᵀ = P (symmetric)
3. Eigenvalues are 0 or 1 only
4. rank(P) = dimension of subspace

ML Applications:
- Linear regression: ŷ = Py
- PCA: project onto principal components
- Removing components (I - P)
"""

print("="*60)
print("Projection onto a Line")
print("="*60)

# Define a line (1D subspace) in ℝ²
a = np.array([[3], [1]])  # Direction vector (column)

print(f"\nLine direction: a = {a.ravel()}")

# Projection matrix onto line spanned by a
# P = a(aᵀa)⁻¹aᵀ = (aaᵀ)/(aᵀa)
P = (a @ a.T) / (a.T @ a)

print(f"\nProjection matrix P:")
print(P)

# Verify properties
print("\n" + "-"*60)
print("Verifying Projection Properties")
print("-"*60)

# Property 1: P² = P
P_squared = P @ P
print(f"\n1. Idempotence: P² = P")
print(f"   P²:")
print(f"   {P_squared}")
print(f"   P:")
print(f"   {P}")
print(f"   Equal? {np.allclose(P_squared, P)} ✓")

# Property 2: Pᵀ = P
P_transpose = P.T
print(f"\n2. Symmetry: Pᵀ = P")
print(f"   Pᵀ:")
print(f"   {P_transpose}")
print(f"   Equal? {np.allclose(P_transpose, P)} ✓")

# Project a vector
print("\n" + "-"*60)
print("Projecting Vectors")
print("-"*60)

v = np.array([[4], [3]])
v_proj = P @ v

print(f"\nOriginal vector v: {v.ravel()}")
print(f"Projected vector Pv: {v_proj.ravel()}")

# The projected vector should lie on the line
# Check if v_proj is parallel to a
# v_proj = scalar * a
scalar = v_proj[0, 0] / a[0, 0]
print(f"\nPv = {scalar:.4f} × a")
print(f"Verify: {scalar:.4f} × {a.ravel()} = {(scalar * a).ravel()}")
print(f"Match? {np.allclose(v_proj, scalar * a)} ✓")

# Perpendicular component
v_perp = v - v_proj
print(f"\nPerpendicular component: v - Pv = {v_perp.ravel()}")

# Check orthogonality: (v - Pv) ⊥ Pv
dot_product = (v_perp.T @ v_proj)[0, 0]
print(f"\nOrthogonality check:")
print(f"(v - Pv) · Pv = {dot_product:.10f}")
print(f"Is orthogonal? {np.isclose(dot_product, 0)} ✓")

# Projection onto a Plane
print("\n" + "="*60)
print("Projection onto a Plane in ℝ³")
print("="*60)

# Define plane spanned by two vectors
a1 = np.array([[1], [0], [0]])
a2 = np.array([[0], [1], [0]])
A = np.hstack([a1, a2])

print(f"\nPlane spanned by:")
print(f"a₁ = {a1.ravel()}")
print(f"a₂ = {a2.ravel()}")
print(f"\nMatrix A = [a₁ a₂]:")
print(A)

# Projection matrix: P = A(AᵀA)⁻¹Aᵀ
ATA = A.T @ A
ATA_inv = np.linalg.inv(ATA)
P_plane = A @ ATA_inv @ A.T

print(f"\nProjection matrix P:")
print(P_plane)

# Project a vector onto the xy-plane
v = np.array([[2], [3], [5]])
v_proj = P_plane @ v

print(f"\nVector v = {v.ravel()}")
print(f"Projection Pv = {v_proj.ravel()}")
print(f"\nNote: z-component is removed (projected onto xy-plane) ✓")

# Verify properties
print(f"\nVerify P² = P:")
print(f"Equal? {np.allclose(P_plane @ P_plane, P_plane)} ✓")

# ML Application: Linear Regression
print("\n" + "="*60)
print("ML Application: Linear Regression as Projection")
print("="*60)

# Generate synthetic data
np.random.seed(42)
n = 50
X = np.random.randn(n, 2)  # 2 features
true_weights = np.array([3, -2])
y = X @ true_weights + np.random.randn(n) * 0.5

print(f"\nData:")
print(f"  X shape: {X.shape}")
print(f"  y shape: {y.shape}")
print(f"  True weights: {true_weights}")

# Linear regression: minimize ||y - Xβ||²
# Solution: β = (XᵀX)⁻¹Xᵀy
beta = np.linalg.inv(X.T @ X) @ X.T @ y

print(f"\nEstimated weights β: {beta}")
print(f"Close to true? {np.allclose(beta, true_weights, atol=0.1)} ✓")

# Projection matrix (hat matrix)
P_hat = X @ np.linalg.inv(X.T @ X) @ X.T

print(f"\nHat matrix P (projection matrix):")
print(f"Shape: {P_hat.shape}")

# Predicted values
y_pred = P_hat @ y  # Same as X @ beta

print(f"\nPredictions:")
print(f"ŷ = Py (projection of y onto column space of X)")
print(f"First 5 predictions: {y_pred[:5]}")
print(f"First 5 actual: {y[:5]}")

# Residuals
residuals = y - y_pred
print(f"\nResiduals: e = y - ŷ = (I - P)y")
print(f"First 5 residuals: {residuals[:5]}")

# Verify orthogonality: Xᵀe = 0
orthogonality_check = X.T @ residuals
print(f"\nOrthogonality: Xᵀe should be ≈ 0")
print(f"Xᵀe = {orthogonality_check}")
print(f"All ≈ 0? {np.allclose(orthogonality_check, 0)} ✓")

print(f"\n→ Linear regression IS a projection!")
print(f"→ Projects y onto space spanned by X columns")
print(f"→ Residuals perpendicular to column space")

Projection onto a Line

Line direction: a = [3 1]

Projection matrix P:
[[0.9 0.3]
 [0.3 0.1]]

------------------------------------------------------------
Verifying Projection Properties
------------------------------------------------------------

1. Idempotence: P² = P
   P²:
   [[0.9 0.3]
 [0.3 0.1]]
   P:
   [[0.9 0.3]
 [0.3 0.1]]
   Equal? True ✓

2. Symmetry: Pᵀ = P
   Pᵀ:
   [[0.9 0.3]
 [0.3 0.1]]
   Equal? True ✓

------------------------------------------------------------
Projecting Vectors
------------------------------------------------------------

Original vector v: [4 3]
Projected vector Pv: [4.5 1.5]

Pv = 1.5000 × a
Verify: 1.5000 × [3 1] = [4.5 1.5]
Match? True ✓

Perpendicular component: v - Pv = [-0.5  1.5]

Orthogonality check:
(v - Pv) · Pv = 0.0000000000
Is orthogonal? True ✓

Projection onto a Plane in ℝ³

Plane spanned by:
a₁ = [1 0 0]
a₂ = [0 1 0]

Matrix A = [a₁ a₂]:
[[1 0]
 [0 1]
 [0 0]]

Projection matrix P:
[[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 0.]]

Vector v 

## 25. Orthogonality & Orthogonal Matrices

Orthogonal vectors are perpendicular (dot product = 0).

**Orthogonal Matrix:** QᵀQ = QQᵀ = I (columns are orthonormal)

**ML Application:** QR decomposition, rotations, whitening transformations.

In [3]:
"""
Orthogonality and Orthogonal Matrices

ORTHOGONAL VECTORS:
- u ⊥ v if u · v = 0
- Perpendicular, at 90° angle

ORTHONORMAL VECTORS:
- Orthogonal AND unit length
- u · v = 0 (different) and ||u|| = ||v|| = 1

ORTHOGONAL MATRIX Q:
- Columns are orthonormal
- QᵀQ = I
- Q⁻¹ = Qᵀ (inverse = transpose!)
- Preserves lengths and angles
- Represents rotation/reflection

Properties:
- ||Qx|| = ||x|| (preserves length)
- (Qx) · (Qy) = x · y (preserves dot products)
- det(Q) = ±1
"""

print("="*60)
print("Orthogonal Vectors")
print("="*60)

# Example 1: Standard basis (orthogonal)
e1 = np.array([1, 0, 0])
e2 = np.array([0, 1, 0])
e3 = np.array([0, 0, 1])

print(f"\nStandard basis vectors:")
print(f"e₁ = {e1}")
print(f"e₂ = {e2}")
print(f"e₃ = {e3}")

# Check pairwise orthogonality
print(f"\nPairwise dot products:")
print(f"e₁ · e₂ = {np.dot(e1, e2)}")
print(f"e₁ · e₃ = {np.dot(e1, e3)}")
print(f"e₂ · e₃ = {np.dot(e2, e3)}")
print(f"→ All zero: vectors are ORTHOGONAL ✓")

# Check unit length
print(f"\nLengths:")
print(f"||e₁|| = {np.linalg.norm(e1)}")
print(f"||e₂|| = {np.linalg.norm(e2)}")
print(f"||e₃|| = {np.linalg.norm(e3)}")
print(f"→ All length 1: vectors are ORTHONORMAL ✓")

# Example 2: Non-orthogonal vectors
print("\n" + "="*60)
print("Non-Orthogonal Vectors")
print("="*60)

u = np.array([1, 2])
v = np.array([3, 1])

dot_uv = np.dot(u, v)

print(f"\nu = {u}")
print(f"v = {v}")
print(f"\nu · v = {dot_uv}")
print(f"→ Not zero: vectors are NOT orthogonal ✗")

# Compute angle
cos_theta = dot_uv / (np.linalg.norm(u) * np.linalg.norm(v))
theta_deg = np.degrees(np.arccos(cos_theta))
print(f"\nAngle between vectors: {theta_deg:.2f}°")

# Gram-Schmidt: Make them orthogonal
print("\n" + "-"*60)
print("Gram-Schmidt Orthogonalization")
print("-"*60)

# Keep u as is
u_orth = u

# Make v orthogonal to u
# v_orth = v - projection of v onto u
projection = (np.dot(v, u) / np.dot(u, u)) * u
v_orth = v - projection

print(f"\nOriginal:")
print(f"u = {u}")
print(f"v = {v}")

print(f"\nOrthogonalized:")
print(f"u_orth = {u_orth}")
print(f"v_orth = {v_orth}")

# Verify orthogonality
dot_orth = np.dot(u_orth, v_orth)
print(f"\nu_orth · v_orth = {dot_orth:.10f}")
print(f"Is orthogonal? {np.isclose(dot_orth, 0)} ✓")

# Normalize to make orthonormal
u_orthonormal = u_orth / np.linalg.norm(u_orth)
v_orthonormal = v_orth / np.linalg.norm(v_orth)

print(f"\nNormalized (orthonormal):")
print(f"û = {u_orthonormal}")
print(f"v̂ = {v_orthonormal}")
print(f"\n||û|| = {np.linalg.norm(u_orthonormal):.4f}")
print(f"||v̂|| = {np.linalg.norm(v_orthonormal):.4f}")
print(f"û · v̂ = {np.dot(u_orthonormal, v_orthonormal):.10f}")
print(f"→ Orthonormal! ✓")

# Orthogonal Matrices
print("\n" + "="*60)
print("Orthogonal Matrix (Rotation)")
print("="*60)

# Rotation matrix (45° in 2D)
theta = np.pi / 4  # 45 degrees
Q = np.array([
    [np.cos(theta), -np.sin(theta)],
    [np.sin(theta),  np.cos(theta)]
])

print(f"\nRotation matrix Q (45°):")
print(Q)

# Verify QᵀQ = I
QTQ = Q.T @ Q
I = np.eye(2)

print(f"\nQᵀQ:")
print(QTQ)
print(f"\nIdentity I:")
print(I)
print(f"\nQᵀQ = I? {np.allclose(QTQ, I)} ✓")

# Verify Q⁻¹ = Qᵀ
Q_inv = np.linalg.inv(Q)
print(f"\nQ⁻¹:")
print(Q_inv)
print(f"\nQᵀ:")
print(Q.T)
print(f"\nQ⁻¹ = Qᵀ? {np.allclose(Q_inv, Q.T)} ✓")

# Properties
print("\n" + "-"*60)
print("Properties of Orthogonal Matrices")
print("-"*60)

# Property 1: Preserves length
x = np.array([3, 4])
Qx = Q @ x

norm_x = np.linalg.norm(x)
norm_Qx = np.linalg.norm(Qx)

print(f"\n1. Length preservation:")
print(f"   x = {x}")
print(f"   Qx = {Qx}")
print(f"   ||x|| = {norm_x:.4f}")
print(f"   ||Qx|| = {norm_Qx:.4f}")
print(f"   Equal? {np.isclose(norm_x, norm_Qx)} ✓")

# Property 2: Determinant = ±1
det_Q = np.linalg.det(Q)
print(f"\n2. Determinant:")
print(f"   det(Q) = {det_Q:.4f}")
print(f"   |det(Q)| = 1? {np.isclose(abs(det_Q), 1)} ✓")
print(f"   → Rotation (det=1) preserves orientation")

# Property 3: Orthonormal columns
col1 = Q[:, 0]
col2 = Q[:, 1]

print(f"\n3. Orthonormal columns:")
print(f"   Column 1: {col1}")
print(f"   Column 2: {col2}")
print(f"   col1 · col2 = {np.dot(col1, col2):.10f} (≈ 0) ✓")
print(f"   ||col1|| = {np.linalg.norm(col1):.4f} (= 1) ✓")
print(f"   ||col2|| = {np.linalg.norm(col2):.4f} (= 1) ✓")

# ML Application: Whitening Transform
print("\n" + "="*60)
print("ML Application: Whitening (Decorrelation)")
print("="*60)

# Generate correlated data
np.random.seed(42)
n_samples = 500
mean = [0, 0]
cov = [[2, 1.5],   # Correlated
       [1.5, 1]]

X = np.random.multivariate_normal(mean, cov, n_samples)

print(f"\nOriginal data covariance:")
cov_original = np.cov(X.T)
print(cov_original)
print(f"→ Features are correlated (off-diagonal ≠ 0)")

# Whiten using eigendecomposition
# Cov = QΛQᵀ
eigenvalues, Q = np.linalg.eigh(cov_original)
Lambda = np.diag(eigenvalues)

# Whitening matrix: W = Λ^(-1/2) Qᵀ
Lambda_inv_sqrt = np.diag(1.0 / np.sqrt(eigenvalues))
W = Lambda_inv_sqrt @ Q.T

# Whiten the data
X_white = (W @ X.T).T

print(f"\nWhitened data covariance:")
cov_white = np.cov(X_white.T)
print(cov_white)
print(f"→ Identity matrix! Features decorrelated ✓")

print(f"\nVerify it's identity:")
print(f"Close to I? {np.allclose(cov_white, np.eye(2), atol=0.1)} ✓")

print(f"\n→ Whitening uses orthogonal transformations")
print(f"→ Decorrelates features")
print(f"→ Used in preprocessing for ML models")

Orthogonal Vectors

Standard basis vectors:
e₁ = [1 0 0]
e₂ = [0 1 0]
e₃ = [0 0 1]

Pairwise dot products:
e₁ · e₂ = 0
e₁ · e₃ = 0
e₂ · e₃ = 0
→ All zero: vectors are ORTHOGONAL ✓

Lengths:
||e₁|| = 1.0
||e₂|| = 1.0
||e₃|| = 1.0
→ All length 1: vectors are ORTHONORMAL ✓

Non-Orthogonal Vectors

u = [1 2]
v = [3 1]

u · v = 5
→ Not zero: vectors are NOT orthogonal ✗

Angle between vectors: 45.00°

------------------------------------------------------------
Gram-Schmidt Orthogonalization
------------------------------------------------------------

Original:
u = [1 2]
v = [3 1]

Orthogonalized:
u_orth = [1 2]
v_orth = [ 2. -1.]

u_orth · v_orth = 0.0000000000
Is orthogonal? True ✓

Normalized (orthonormal):
û = [0.4472 0.8944]
v̂ = [ 0.8944 -0.4472]

||û|| = 1.0000
||v̂|| = 1.0000
û · v̂ = 0.0000000000
→ Orthonormal! ✓

Orthogonal Matrix (Rotation)

Rotation matrix Q (45°):
[[ 0.7071 -0.7071]
 [ 0.7071  0.7071]]

QᵀQ:
[[ 1. -0.]
 [-0.  1.]]

Identity I:
[[1. 0.]
 [0. 1.]]

QᵀQ = I? True

C:\Users\Aamir Bin Abbas\AppData\Local\Temp\ipykernel_2092\2295758346.py:193: RuntimeWarning: covariance is not symmetric positive-semidefinite.
  X = np.random.multivariate_normal(mean, cov, n_samples)


## 26. Eigenvalues & Eigenvectors

For square matrix A, if Av = λv (v ≠ 0), then:
- v is an **eigenvector**
- λ is an **eigenvalue**

**Interpretation:** Direction that only gets scaled, not rotated.

**ML Application:** PCA, spectral clustering, PageRank, stability analysis.

In [ ]:
"""
Eigenvalues and Eigenvectors

Definition:
For square matrix A, a non-zero vector v is an eigenvector if:
    Av = λv

where λ is a scalar (the eigenvalue)

Interpretation:
- Av is parallel to v (just scaled)
- Direction v is preserved under transformation A
- λ tells how much v is stretched/compressed
  - |λ| > 1: stretches
  - |λ| < 1: compresses  
  - λ < 0: flips direction

Computing:
1. Solve det(A - λI) = 0 for eigenvalues λ
2. For each λ, solve (A - λI)v = 0 for eigenvector v

ML Applications:
- PCA: eigenvectors = principal components
- Eigenvalues = variance along components
- Spectral clustering
- Stability of dynamical systems (RNNs)
"""

print("="*60)
print("Eigenvalues and Eigenvectors")
print("="*60)

# Simple 2×2 example
A = np.array([
    [4, 2],
    [1, 3]
])

print(f"\nMatrix A:")
print(A)

# Compute eigenvalues and eigenvectors
eigenvalues, eigenvectors = np.linalg.eig(A)

print(f"\nEigenvalues λ:")
print(eigenvalues)

print(f"\nEigenvectors (as columns):")
print(eigenvectors)

# Verify for each eigenvalue/eigenvector pair
print("\n" + "-"*60)
print("Verification: Av = λv")
print("-"*60)

for i in range(len(eigenvalues)):
    λ = eigenvalues[i]
    v = eigenvectors[:, i]
    
    Av = A @ v
    λv = λ * v
    
    print(f"\nEigenpair {i+1}:")
    print(f"  λ = {λ:.4f}")
    print(f"  v = {v}")
    print(f"  Av = {Av}")
    print(f"  λv = {λv}")
    print(f"  Av = λv? {np.allclose(Av, λv)} ✓")

# Geometric interpretation
print("\n" + "="*60)
print("Geometric Interpretation")
print("="*60)

print(f"\nEigenvector 1 direction: {eigenvectors[:, 0]}")
print(f"Eigenvalue 1: {eigenvalues[0]:.4f}")
if abs(eigenvalues[0]) > 1:
    print(f"→ Vector gets STRETCHED by factor {abs(eigenvalues[0]):.4f}")
else:
    print(f"→ Vector gets COMPRESSED by factor {abs(eigenvalues[0]):.4f}")

print(f"\nEigenvector 2 direction: {eigenvectors[:, 1]}")
print(f"Eigenvalue 2: {eigenvalues[1]:.4f}")
if abs(eigenvalues[1]) > 1:
    print(f"→ Vector gets STRETCHED by factor {abs(eigenvalues[1]):.4f}")
else:
    print(f"→ Vector gets COMPRESSED by factor {abs(eigenvalues[1]):.4f}")

# Eigendecomposition
print("\n" + "="*60)
print("Eigendecomposition: A = QΛQ⁻¹")
print("="*60)

Q = eigenvectors  # Matrix of eigenvectors
Lambda = np.diag(eigenvalues)  # Diagonal matrix of eigenvalues

print(f"\nQ (eigenvectors as columns):")
print(Q)

print(f"\nΛ (eigenvalues on diagonal):")
print(Lambda)

# Reconstruct A
Q_inv = np.linalg.inv(Q)
A_reconstructed = Q @ Lambda @ Q_inv

print(f"\nReconstructed A = QΛQ⁻¹:")
print(A_reconstructed)

print(f"\nOriginal A:")
print(A)

print(f"\nMatch? {np.allclose(A_reconstructed, A)} ✓")

# Power of eigendecomposition: Easy to compute A^n
print("\n" + "-"*60)
print("Computing Matrix Powers: A^n = QΛ^nQ⁻¹")
print("-"*60)

n = 10

# Method 1: Direct (slow for large n)
A_power_direct = np.linalg.matrix_power(A, n)

# Method 2: Using eigendecomposition (fast!)
Lambda_n = np.diag(eigenvalues**n)  # Just raise eigenvalues to power n
A_power_eigen = Q @ Lambda_n @ Q_inv

print(f"\nA^{n} (direct):")
print(A_power_direct)

print(f"\nA^{n} (eigendecomposition):")
print(A_power_eigen)

print(f"\nMatch? {np.allclose(A_power_direct, A_power_eigen)} ✓")

print(f"\n→ Eigendecomposition makes powers easy!")
print(f"→ Just raise diagonal eigenvalues to power")

# Special matrices
print("\n" + "="*60)
print("Symmetric Matrices: Special Properties")
print("="*60)

# Symmetric matrix (like covariance)
S = np.array([
    [4, 1],
    [1, 3]
])

print(f"\nSymmetric matrix S:")
print(S)
print(f"S = Sᵀ? {np.allclose(S, S.T)} ✓")

eigenvalues_sym, eigenvectors_sym = np.linalg.eigh(S)  # Use eigh for symmetric

print(f"\nEigenvalues:")
print(eigenvalues_sym)
print(f"→ All REAL (guaranteed for symmetric) ✓")

# Check orthogonality of eigenvectors
v1 = eigenvectors_sym[:, 0]
v2 = eigenvectors_sym[:, 1]
dot_v1_v2 = np.dot(v1, v2)

print(f"\nEigenvectors:")
print(f"v₁ = {v1}")
print(f"v₂ = {v2}")
print(f"v₁ · v₂ = {dot_v1_v2:.10f}")
print(f"→ ORTHOGONAL (guaranteed for symmetric) ✓")

print(f"\nFor symmetric matrices:")
print(f"1. Eigenvalues are real")
print(f"2. Eigenvectors are orthogonal")
print(f"3. Can write S = QΛQᵀ (Q is orthogonal!)")

# ML Application: PCA
print("\n" + "="*60)
print("ML Application: PCA via Eigendecomposition")
print("="*60)

# Generate data
np.random.seed(42)
n_samples = 200
mean = [0, 0]
cov = [[3, 1.5],
       [1.5, 1]]

X = np.random.multivariate_normal(mean, cov, n_samples)

print(f"\nData shape: {X.shape}")

# Center the data
X_centered = X - X.mean(axis=0)

# Compute covariance matrix
C = np.cov(X_centered.T)

print(f"\nCovariance matrix:")
print(C)

# Eigendecomposition of covariance
eigenvalues_pca, eigenvectors_pca = np.linalg.eigh(C)

# Sort by eigenvalue (descending)
idx = eigenvalues_pca.argsort()[::-1]
eigenvalues_pca = eigenvalues_pca[idx]
eigenvectors_pca = eigenvectors_pca[:, idx]

print(f"\nPrincipal Components (eigenvectors):")
print(f"PC1: {eigenvectors_pca[:, 0]}")
print(f"PC2: {eigenvectors_pca[:, 1]}")

print(f"\nVariance explained (eigenvalues):")
print(f"PC1: {eigenvalues_pca[0]:.4f}")
print(f"PC2: {eigenvalues_pca[1]:.4f}")

# Variance explained ratio
total_variance = eigenvalues_pca.sum()
explained_variance_ratio = eigenvalues_pca / total_variance

print(f"\nVariance explained ratio:")
print(f"PC1: {explained_variance_ratio[0]*100:.2f}%")
print(f"PC2: {explained_variance_ratio[1]*100:.2f}%")

# Project data onto principal components
X_pca = X_centered @ eigenvectors_pca

print(f"\nTransformed data shape: {X_pca.shape}")

# Verify: variance in PC directions
var_pc1 = np.var(X_pca[:, 0])
var_pc2 = np.var(X_pca[:, 1])

print(f"\nVerification:")
print(f"Variance in PC1 direction: {var_pc1:.4f}")
print(f"Eigenvalue λ₁: {eigenvalues_pca[0]:.4f}")
print(f"Match? {np.isclose(var_pc1, eigenvalues_pca[0])} ✓")

print(f"\n→ PCA finds directions of maximum variance")
print(f"→ These directions are eigenvectors of covariance")
print(f"→ Variance along each = corresponding eigenvalue")
print(f"→ Dimensionality reduction: keep top k components")

# Linear Algebra for AI/ML - Part 4: Summary Table

| Concept | Key Formula(s) | Properties | ML Application |
|---------|----------------|------------|----------------|
| Projection Matrix | P = A(AᵀA)⁻¹Aᵀ | Idempotent (P² = P), Symmetric (Pᵀ = P) | Linear regression (ŷ = Py), PCA, feature projection |
| Orthogonal Vectors | u · v = 0 | Perpendicular, length preserved under orthogonal transforms | Decorrelation, whitening, Gram–Schmidt orthogonalization |
| Orthogonal Matrix | QᵀQ = I, Q⁻¹ = Qᵀ | Columns orthonormal, preserves length & angles, det(Q) = ±1 | Rotations, QR decomposition, whitening transforms |
| Eigenvalues & Eigenvectors | Av = λv | Eigenvectors invariant under transformation, eigenvalues scale | PCA, spectral clustering, stability analysis, matrix powers |
| Eigendecomposition | A = QΛQ⁻¹ | For symmetric matrices: eigenvalues real, eigenvectors orthogonal | Principal Component Analysis (PCA), variance analysis |
| PCA via Covariance | C = XᵀX/(n-1) | Eigenvectors = principal components, eigenvalues = variance | Dimensionality reduction, feature extraction, data visualization |

## Part 4 Complete!

**Completed:**
- Projection Matrices (24)
- Orthogonality & Orthogonal Matrices (25)
- Eigenvalues & Eigenvectors (26)

**Part 5 will cover:**
- SVD (Singular Value Decomposition)
- Matrix Decompositions (Cholesky, QR, LU)
- Trace & Determinant properties
- Covariance Matrix.
